In [10]:
import requests                  # [handles the http interactions](http://docs.python-requests.org/en/master/) 
from bs4 import BeautifulSoup    # beautiful soup handles the html to text conversion and more
import re                        # regular expressions are necessary for finding the crumb (more on crumbs later)
from datetime import timedelta,datetime    # string to datetime object conversion
from time import mktime          # mktime transforms datetime objects to unix timestamps

########################################################
def _get_crumbs_and_cookies(stock):
    """
    get crumb and cookies for historical data csv download from yahoo finance
    
    parameters: stock - short-handle identifier of the company 
    
    returns a tuple of header, crumb and cookie
    """
    
    url = 'https://finance.yahoo.com/quote/{}/history'.format(stock)
    with requests.session():
        header = {'Connection': 'keep-alive',
                   'Expires': '-1',
                   'Upgrade-Insecure-Requests': '1',
                   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) \
                   AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36'
                   }
        
        website = requests.get(url, headers=header)
        soup = BeautifulSoup(website.text, 'lxml')
        crumb = re.findall('"CrumbStore":{"crumb":"(.+?)"}', str(soup))

        return (header, crumb[0], website.cookies)


########################################################
def convert_to_unix(date):
    """
    converts date to unix timestamp
    
    parameters: date - in format (dd-mm-yyyy)
    
    returns integer unix timestamp
    """
    datum = datetime.strptime(date, '%d-%m-%Y')
    
    return int(mktime(datum.timetuple()))


########################################################
def load_csv_data(stock, interval='1d', day_begin=(datetime.now()-timedelta(days=14)).strftime('%d-%m-%Y'), day_end=datetime.now().strftime('%d-%m-%Y')):
    """
    queries yahoo finance api to receive historical data in csv file format
    
    parameters: 
        stock - short-handle identifier of the company
        
        interval - 1d, 1wk, 1mo - daily, weekly monthly data
        
        day_begin - starting date for the historical data (format: dd-mm-yyyy)
        
        day_end - final date of the data (format: dd-mm-yyyy)
    
    returns a list of comma seperated value lines
    """
    day_begin_unix = convert_to_unix(day_begin)
    day_end_unix = convert_to_unix(day_end)
    
    header, crumb, cookies = _get_crumbs_and_cookies(stock)
    
    with requests.session():
        url = 'https://query1.finance.yahoo.com/v7/finance/download/' \
              '{stock}?period1={day_begin}&period2={day_end}&interval={interval}&events=history&crumb={crumb}' \
              .format(stock=stock, day_begin=day_begin_unix, day_end=day_end_unix, interval=interval, crumb=crumb)
                
        website = requests.get(url, headers=header, cookies=cookies)
       
        return website.text.split('\n')[:-1]


########################################################
csv = load_csv_data('0941.HK')
for c in csv:
    print c

Date,Open,High,Low,Close,Adj Close,Volume
2019-01-14,80.150002,80.500000,79.550003,80.050003,80.050003,14885907
2019-01-15,80.150002,82.150002,80.050003,81.599998,81.599998,24936098
2019-01-16,80.900002,81.400002,80.099998,81.400002,81.400002,15471447
2019-01-17,81.599998,81.599998,80.250000,80.650002,80.650002,15597142
2019-01-18,80.650002,81.400002,80.500000,80.849998,80.849998,15945592
2019-01-21,80.849998,80.949997,80.449997,80.949997,80.949997,13120812
2019-01-22,80.400002,80.949997,80.000000,80.349998,80.349998,18843047
2019-01-23,79.800003,81.550003,79.800003,81.300003,81.300003,23507897
2019-01-24,81.699997,82.449997,81.300003,82.349998,82.349998,23707991
2019-01-25,81.800003,82.349998,80.849998,81.150002,81.150002,24080890
